In [38]:
import os
import sys
from astropy.table import Table
from desisurvey.progress import Progress
import desimodel.io
import numpy as np
import datetime
import yaml


In [30]:
os.environ['DESISURVEY_OUTPUT'] = os.path.join('/global/cscratch1/sd/forero/quicksurvey2017b/survey')

In [70]:
def compute_fiberassign_dates(exposurefile):
    desisurvey_path = os.getenv('DESISURVEY_OUTPUT')
    exposures = Table.read(exposurefile)
    expdates = []
    for n in exposures['night']:
        a = datetime.datetime.strptime(n, "%Y-%m-%d")
        expdates.append(a.date())
    expdates = np.array(expdates)

    # compute the dates for fiberassign
    progress = Table.read(os.path.join(desisurvey_path,'progress.fits'))
    fiberassign_dates = np.sort(list(set(progress['available'])))
    fiberassign_dates = fiberassign_dates[fiberassign_dates>0]
    
    #load the first date of the survey
    survey_config = os.path.join(desisurvey_path,'config.yaml')
    with open(survey_config, 'r') as pfile:
        params = yaml.load(pfile)
    pfile.close()
    first_day = params['first_day']
    last_day = params['last_day']
    print(first_day, last_day)
    
    #compute the dates to run fiberassign to be sure that there is at least one exposure there
    fiberassign_dates_run = []
    one_day = datetime.timedelta(days=1)
    initial_day = first_day 
    for d in fiberassign_dates:
        final_day = first_day + d * one_day
        ii = (expdates > initial_day) & (expdates<=final_day)
        n_in = np.count_nonzero(ii)
        if n_in>0:
            print(initial_day, final_day, n_in)
            fiberassign_dates_run.append(initial_day.strftime("%Y-%m-%d"))
            initial_day = final_day
    return fiberassign_dates_run

desisurvey_path = os.getenv('DESISURVEY_OUTPUT')

if desisurvey_path is None:
    print('Environment variable DESISURVEY_OUTPUT is not set')
else:
    print('Environment variable DESISURVEY_OUTPUT set to {}'.format(desisurvey_path))
    Progress(restore='progress.fits').get_exposures().write(os.path.join(desisurvey_path,'exposures.fits'), overwrite=True)
    
    explist = Table.read(os.path.join(desisurvey_path,'exposures.fits'))

    # separate the exposures for dark and bright programs
    isbright = explist['pass'] > 4 
    isgray = explist['pass'] == 4
    isdark = explist['pass'] < 4
    
    exposurefile_bright = os.path.join(desisurvey_path,'exposures_bright.fits')
    Table(explist[isbright]).write(exposurefile_bright, overwrite=True)

    exposurefile_dark = os.path.join(desisurvey_path,'exposures_dark.fits')
    Table(explist[~isbright]).write(exposurefile_dark, overwrite=True)
    

    fiberdatesfile_bright = os.path.join(desisurvey_path, 'fiberassign_dates_bright.txt')
    fiberassign_dates_bright = compute_fiberassign_dates(exposurefile_bright)
    f = open(fiberdatesfile_bright, 'w')
    for d in fiberassign_dates_bright:
        f.write(d+"\n")
    f.close()
    
    fiberdatesfile_dark = os.path.join(desisurvey_path, 'fiberassign_dates_dark.txt')
    fiberassign_dates_dark = compute_fiberassign_dates(exposurefile_dark)
    f = open(fiberdatesfile_dark, 'w')
    for d in fiberassign_dates_dark:
        f.write(d+"\n")
    f.close()



Environment variable DESISURVEY_OUTPUT set to /global/cscratch1/sd/forero/quicksurvey2017b/survey
INFO:progress.py:145:__init__: Loaded progress from /global/cscratch1/sd/forero/quicksurvey2017b/survey/progress.fits.


2019-12-01 2024-11-30
2019-12-01 2020-06-05 1262
2020-06-05 2020-07-04 177
2020-07-04 2020-08-02 20
2020-08-02 2020-10-01 138
2020-10-01 2020-10-30 194
2020-10-30 2020-11-29 213
2020-11-29 2020-12-29 178
2020-12-29 2021-01-27 147
2021-01-27 2021-03-27 230
2021-03-27 2021-04-26 201
2021-04-26 2021-05-25 98
2021-05-25 2021-06-23 57
2021-06-23 2021-07-23 39
2021-07-23 2021-09-20 122
2021-09-20 2021-10-19 284
2021-10-19 2021-11-18 231
2021-11-18 2021-12-18 129
2021-12-18 2022-01-17 126
2022-01-17 2022-03-17 485
2022-03-17 2022-04-15 241
2022-04-15 2022-05-15 53
2022-05-15 2022-06-13 39
2022-06-13 2022-07-12 12
2022-07-12 2022-09-09 42
2022-09-09 2022-10-09 156
2022-10-09 2022-11-07 197
2022-11-07 2022-12-07 327
2022-12-07 2023-01-06 418
2023-01-06 2023-03-06 453
2023-03-06 2023-04-05 222
2023-04-05 2023-05-04 45
2023-05-04 2023-06-03 40
2023-06-03 2023-07-02 43
2023-07-02 2023-07-31 9
2023-07-31 2023-09-28 53
2019-12-01 2024-11-30
2019-12-01 2020-06-05 2994
2020-06-05 2020-07-04 317
2020-0

In [43]:
print('Environment variable DESISURVEY_OUTPUT set to {}'.format(desisurvey_path))
Progress(restore='progress.fits').get_exposures().write(os.path.join(desisurvey_path,'exposures.fits'), overwrite=True)
    
explist = Table.read(os.path.join(desisurvey_path,'exposures.fits'))

    # separate the exposures for dark and bright programs
isbright = explist['pass'] > 4 
isgray = explist['pass'] == 4
isdark = explist['pass'] < 4

Environment variable DESISURVEY_OUTPUT set to /global/cscratch1/sd/forero/quicksurvey2017b/survey
INFO:progress.py:145:__init__: Loaded progress from /global/cscratch1/sd/forero/quicksurvey2017b/survey/progress.fits.


In [62]:
exposurefile_bright = os.path.join(desisurvey_path,'exposures_bright.fits')
Table(explist[isbright]).write(exposurefile_bright, overwrite=True)

exposurefile_dark = os.path.join(desisurvey_path,'exposures_dark.fits')
Table(explist[~isbright]).write(exposurefile_dark, overwrite=True)
    

In [63]:
#compute the dates for the bright exposures
exposures = Table.read(exposurefile_bright)
expdates = []
for n in exposures['night']:
    a = datetime.datetime.strptime(n, "%Y-%m-%d")
    expdates.append(a.date())
expdates = np.array(expdates)

In [64]:
expdates

array([datetime.date(2019, 12, 1), datetime.date(2019, 12, 1),
       datetime.date(2019, 12, 1), ..., datetime.date(2024, 6, 6),
       datetime.date(2024, 6, 8), datetime.date(2024, 8, 28)], dtype=object)

In [65]:
survey_config = os.path.join(desisurvey_path,'config.yaml')
with open(survey_config, 'r') as pfile:
    params = yaml.load(pfile)
pfile.close()
first_day = params['first_day']
last_day = params['last_day']
print(first_day, last_day)

2019-12-01 2024-11-30


In [68]:
#compute the dates to run fiberassign to be sure that there is at least one exposure there
fiberassign_dates = []
one_day = datetime.timedelta(days=1)
initial_day = first_day 
for d in fiberassign_dates:
    final_day = first_day + d * one_day
    print(i)
    ii = (expdates > initial_day) & (expdates<=final_day)
    n_in = np.count_nonzero(ii)
    if n_in>0:
        print(initial_day, final_day, n_in)
        fiberassign_dates.append(initial_day.strftime("%Y-%m-%d"))
        initial_day = final_day

print(fiberassign_dates)

[]
